с=10 и gamma=0.001.
В окресностях этой точки поищим параметры, улучшающий классификатор

CV нашла результат лучше, но на сайте неподтрвердилось.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

x_train = pd.read_csv('data/x_train_norm.csv', sep=';', na_values='?')
x_test = pd.read_csv('data/x_test_norm.csv', sep=';', na_values='?')
y_train = pd.read_csv('data/y_train.csv', sep=';', header=None, na_values='?')[0]

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

def scorer_logloss(estimator, X, y):
    return logloss(y, estimator.predict_proba(X)[:,1])

def revert_scorer_logloss(estimator, X, y):
    return -1 * scorer_logloss(estimator, X, y)

In [2]:
# classifier = SVC(probability=True, C=10, gamma=0.001)
# classifier.fit(x_train, y_train)
# y_test = classifier.predict_proba(x_test)[:,1]  # возвращается думерный массив, нас интересует 2-й стоблец
# res_df = pd.DataFrame(y_test, columns = ['y'])
# res_df.to_csv('res/003_svc_C_10_gamma_0.001.csv', sep=';', header=None, index=False)

In [8]:
C_array = np.linspace(8, 12, num=4) 
gamma_array = np.linspace(0.0005, 0.003, num=4)

svc = SVC(probability=True)
grid = GridSearchCV(svc,
                    param_grid={'C': C_array, 'gamma': gamma_array}, 
                    scoring=revert_scorer_logloss, 
                    n_jobs=3, 
                    verbose=3)
grid.fit(x_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] C=8.0, gamma=0.0005 .............................................
[CV] C=8.0, gamma=0.0005 .............................................
[CV] C=8.0, gamma=0.0005 .............................................
[CV] ............. C=8.0, gamma=0.0005, score=-0.416407, total= 1.6min
[CV] C=8.0, gamma=0.00133333333333 ...................................
[CV] ............. C=8.0, gamma=0.0005, score=-0.413390, total= 1.6min
[CV] C=8.0, gamma=0.00133333333333 ...................................
[CV] ............. C=8.0, gamma=0.0005, score=-0.407320, total= 1.6min
[CV] C=8.0, gamma=0.00133333333333 ...................................
[CV] ... C=8.0, gamma=0.00133333333333, score=-0.404061, total= 1.5min
[CV] C=8.0, gamma=0.00216666666667 ...................................
[CV] ... C=8.0, gamma=0.00133333333333, score=-0.410515, total= 1.5min
[CV] C=8.0, gamma=0.00216666666667 ...................................
[CV] ... C=8.0, 

[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed: 16.3min


[CV] ... C=10.6666666667, gamma=0.0005, score=-0.416294, total= 1.6min
[CV] C=10.6666666667, gamma=0.00133333333333 .........................
[CV]  C=10.6666666667, gamma=0.00133333333333, score=-0.403707, total= 1.5min
[CV] C=10.6666666667, gamma=0.00216666666667 .........................
[CV]  C=10.6666666667, gamma=0.00133333333333, score=-0.410672, total= 1.5min
[CV] C=10.6666666667, gamma=0.00216666666667 .........................
[CV]  C=10.6666666667, gamma=0.00133333333333, score=-0.410283, total= 1.5min
[CV] C=10.6666666667, gamma=0.00216666666667 .........................
[CV]  C=10.6666666667, gamma=0.00216666666667, score=-0.406020, total= 1.8min
[CV] C=10.6666666667, gamma=0.003 ....................................
[CV]  C=10.6666666667, gamma=0.00216666666667, score=-0.411229, total= 1.9min
[CV] C=10.6666666667, gamma=0.003 ....................................
[CV]  C=10.6666666667, gamma=0.00216666666667, score=-0.413865, total= 1.9min
[CV] C=10.6666666667, gamma=0.003 .

[Parallel(n_jobs=3)]: Done  48 out of  48 | elapsed: 28.9min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'C': array([  8.     ,   9.33333,  10.66667,  12.     ]), 'gamma': array([ 0.0005 ,  0.00133,  0.00217,  0.003  ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=<function revert_scorer_logloss at 0x11a00fc08>, verbose=3)

In [11]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_C,param_gamma,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,92.654794,3.630585,-0.412372,-0.411626,8,0.0005,"{u'C': 8.0, u'gamma': 0.0005}",16,-0.407320,-0.412050,-0.413390,-0.413222,-0.416407,-0.409606,1.686826,0.305040,0.003779,0.001506
1,89.931939,3.210112,-0.408265,-0.407056,8,0.00133333,"{u'C': 8.0, u'gamma': 0.00133333333333}",3,-0.404061,-0.406421,-0.410220,-0.408869,-0.410515,-0.405876,1.333708,0.095386,0.002975,0.001302
2,95.327033,3.306280,-0.409972,-0.408397,8,0.00216667,"{u'C': 8.0, u'gamma': 0.00216666666667}",5,-0.405169,-0.406475,-0.413672,-0.411206,-0.411075,-0.407510,0.115092,0.108954,0.003558,0.002031
3,93.307973,3.295284,-0.411289,-0.409456,8,0.003,"{u'C': 8.0, u'gamma': 0.003}",9,-0.407765,-0.408600,-0.414487,-0.411023,-0.411615,-0.408747,0.285893,0.270165,0.002754,0.001109
4,89.435954,3.299053,-0.412317,-0.411552,9.33333,0.0005,"{u'C': 9.33333333333, u'gamma': 0.0005}",15,-0.407152,-0.411735,-0.413453,-0.413262,-0.416347,-0.409657,0.894587,0.067724,0.003839,0.001477
5,96.103382,3.394827,-0.408160,-0.406859,9.33333,0.00133333,"{u'C': 9.33333333333, u'gamma': 0.00133333333333}",1,-0.403774,-0.405900,-0.410369,-0.408856,-0.410337,-0.405820,1.436934,0.051300,0.003101,0.001413
6,122.973443,3.698707,-0.410136,-0.408409,9.33333,0.00216667,"{u'C': 9.33333333333, u'gamma': 0.00216666666667}",6,-0.405616,-0.406775,-0.413640,-0.410776,-0.411154,-0.407677,1.599167,0.014150,0.003354,0.001713
7,112.551738,3.896169,-0.411523,-0.409559,9.33333,0.003,"{u'C': 9.33333333333, u'gamma': 0.003}",10,-0.407941,-0.408573,-0.414909,-0.411261,-0.411718,-0.408845,1.242795,0.041377,0.002848,0.001208
8,93.842938,3.185326,-0.412120,-0.411286,10.6667,0.0005,"{u'C': 10.6666666667, u'gamma': 0.0005}",14,-0.406799,-0.411239,-0.413269,-0.413099,-0.416294,-0.409520,0.749751,0.153106,0.003961,0.001462
9,86.002304,3.207952,-0.408220,-0.406873,10.6667,0.00133333,"{u'C': 10.6666666667, u'gamma': 0.00133333333333}",2,-0.403707,-0.405749,-0.410672,-0.408931,-0.410283,-0.405939,1.211923,0.198188,0.003196,0.001457


In [12]:
C_array = [9, 9.5, 10, 10.5, 11]
gamma_array = [0.0003, 0.0006, 0.001, 0.0013,0.0016]

svc = SVC(probability=True)
grid = GridSearchCV(svc,
                    param_grid={'C': C_array, 'gamma': gamma_array}, 
                    scoring=revert_scorer_logloss, 
                    n_jobs=3, 
                    verbose=3)
grid.fit(x_train, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=9, gamma=0.0003 ...............................................
[CV] C=9, gamma=0.0003 ...............................................
[CV] C=9, gamma=0.0003 ...............................................
[CV] ............... C=9, gamma=0.0003, score=-0.417524, total= 1.6min
[CV] C=9, gamma=0.0006 ...............................................
[CV] ............... C=9, gamma=0.0003, score=-0.409080, total= 1.6min
[CV] C=9, gamma=0.0006 ...............................................
[CV] ............... C=9, gamma=0.0003, score=-0.413842, total= 1.6min
[CV] C=9, gamma=0.0006 ...............................................
[CV] ............... C=9, gamma=0.0006, score=-0.415526, total= 1.6min
[CV] C=9, gamma=0.001 ................................................
[CV] ............... C=9, gamma=0.0006, score=-0.412798, total= 1.6min
[CV] ............... C=9, gamma=0.0006, score=-0.406286, total= 1.6min
[CV] C=9, gamma=

[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed: 17.1min


[CV] ............. C=9.5, gamma=0.0013, score=-0.410308, total= 1.6min
[CV] C=9.5, gamma=0.0016 .............................................
[CV] ............. C=9.5, gamma=0.0016, score=-0.404040, total= 1.4min
[CV] C=10, gamma=0.0003 ..............................................
[CV] ............. C=9.5, gamma=0.0016, score=-0.411959, total= 1.4min
[CV] C=10, gamma=0.0003 ..............................................
[CV] ............. C=9.5, gamma=0.0016, score=-0.410509, total= 1.4min
[CV] C=10, gamma=0.0003 ..............................................
[CV] .............. C=10, gamma=0.0003, score=-0.408810, total= 1.4min
[CV] C=10, gamma=0.0006 ..............................................
[CV] .............. C=10, gamma=0.0003, score=-0.413787, total= 1.4min
[CV] C=10, gamma=0.0006 ..............................................
[CV] .............. C=10, gamma=0.0003, score=-0.417548, total= 1.4min
[CV] C=10, gamma=0.0006 ..............................................
[CV] .

[Parallel(n_jobs=3)]: Done  75 out of  75 | elapsed: 52.5min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'C': [9, 9.5, 10, 10.5, 11], 'gamma': [0.0003, 0.0006, 0.001, 0.0013, 0.0016]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=<function revert_scorer_logloss at 0x11a00fc08>, verbose=3)

In [13]:
grid.best_score_

-0.40810608731638415

In [14]:
grid.best_params_

{'C': 10, 'gamma': 0.0013}

In [15]:
classifier = SVC(probability=True, C=10, gamma=0.0013)
classifier.fit(x_train, y_train)
y_test = classifier.predict_proba(x_test)[:,1]  # возвращается думерный массив, нас интересует 2-й стоблец
res_df = pd.DataFrame(y_test, columns = ['y'])
res_df.to_csv('res/003_svc_C_10_gamma_0.0013.csv', sep=';', header=None, index=False)

#cv = 0.40810608731638415
#bootcamp = 0.4087305